In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [2]:
from qml_hep_lhc.data import MNIST
from qml_hep_lhc.layers.utils import convolution_iters
import argparse

2022-08-07 12:12:36.035475: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-07 12:12:36.035508: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-07 12:12:40.036815: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-07 12:12:40.036856: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-08-07 12:12:40.037218: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [3]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.resize = [8,8]
args.standardize = 1
args.binary_data = [0,1]
args.labels_to_categorical = 1
args.processed = 1
# args.percent_samples = 0.1

# Base Model
args.wandb = False
args.batch_size = 64
args.epochs = 8
args.learning_rate = 0.2


# Quantum CNN Parameters
args.n_layers = 2
args.n_qubits = 9
args.template = 'S2D'

In [4]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤════════════════╤════════════════╤════════════════╤═══════════╕
│ Data   │ Train size     │ Val size       │ Test size      │ Dims      │
╞════════╪════════════════╪════════════════╪════════════════╪═══════════╡
│ X      │ (480, 8, 8, 1) │ (120, 8, 8, 1) │ (200, 8, 8, 1) │ (8, 8, 1) │
├────────┼────────────────┼────────────────┼────────────────┼───────────┤
│ y      │ (480, 2)       │ (120, 2)       │ (200, 2)       │ (2,)      │
╘════════╧════════════════╧════════════════╧════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -1.57 │ 23.52

In [5]:
import tensorflow as tf
import numpy as np
import wandb

tf.keras.backend.set_floatx('float64')


In [6]:
callbacks = []

# LR Scheduler callback
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                          factor=0.1,
                                          patience=5,
                                          min_delta=0.0001,
                                          min_lr=1e-6)
callbacks.append(lr_scheduler_callback)

if args.wandb:
    run_id = wandb.util.generate_id()
    wandb.init(project='qml-hep-lhc',
                       config=vars(args),
                       id=run_id,
                       resume='allow')
    callbacks.append(wandb.keras.WandbCallback(save_weights_only=True, save_graph=False))

wandb: Currently logged in as: gopald. Use `wandb login --relogin` to force relogin
2022-08-07 12:12:44.868290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-07 12:12:44.868333: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [7]:
import pennylane as qml


dev = qml.device("lightning.qubit", wires=args.n_qubits)
qubits = list(range(args.n_qubits))
n_inputs = 9

In [19]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_sparse(inputs, weights, bias):
    z = tf.tensordot(weights, inputs, axes = 1) + bias
    for l in range(args.n_layers):
        for q in qubits:
            qml.Rot(z[l,q,0], z[l,q,1], z[l,q,2], wires= q)
        if (l & 1):
            for q0, q1 in zip(qubits[1::2], qubits[2::2] + [qubits[0]]):
                qml.CZ((q0,q1))
        else:
            for q0, q1 in zip(qubits[0::2], qubits[1::2]):
                qml.CZ((q0,q1))         
    return [qml.expval(qml.PauliZ(i)) for i in range(args.n_qubits)]

In [20]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_basic(inputs, weights, bias):
    inputs = inputs + bias
    qml.AngleEmbedding(inputs, wires=range(args.n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(args.n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(args.n_qubits)]

In [21]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_s2d(inputs, weights, bias):
    z = tf.tensordot(weights, inputs, axes = 1) + bias
    qml.SimplifiedTwoDesign(initial_layer_weights=tf.zeros((args.n_qubits,)), 
                            weights=z, 
                            wires=range(args.n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(args.n_qubits)]

In [22]:
def get_node(template, num_layers, num_qubits, num_inputs):
    if template == 'NQubitPQC':
        return qnode_sparse, {
            "weights": (num_layers, num_qubits, 3, num_inputs),
            "bias": (num_layers, num_qubits, 3)
        }
    elif template == 'Basic':
        assert num_inputs == args.n_qubits
        return qnode_basic, {
            "weights": (num_layers, num_qubits),
            "bias": (num_qubits,)
        }
    elif template == 'S2D':
        return qnode_s2d, {
            "weights": (num_layers, num_qubits -1, 2, num_inputs),
            "bias": (num_layers, num_qubits -1, 2)
        }

In [23]:
inputs = tf.random.uniform((args.n_qubits, ))
node, shapes = get_node(args.template, args.n_layers, args.n_qubits, n_inputs)
w = tf.random.uniform(shapes['weights'])
b = tf.random.uniform(shapes['bias'])
drawer = qml.draw(node, expansion_strategy="device")
print(drawer(inputs,w,b))

0: ──RY(0.00)─╭●──RY(2.73)──────────────╭●──RY(2.90)──────────────┤  <Z>
1: ──RY(0.00)─╰Z──RY(2.81)─╭●──RY(2.59)─╰Z──RY(2.92)─╭●──RY(3.29)─┤  <Z>
2: ──RY(0.00)─╭●──RY(3.15)─╰Z──RY(3.31)─╭●──RY(2.13)─╰Z──RY(2.59)─┤  <Z>
3: ──RY(0.00)─╰Z──RY(1.93)─╭●──RY(2.91)─╰Z──RY(3.43)─╭●──RY(4.10)─┤  <Z>
4: ──RY(0.00)─╭●──RY(3.26)─╰Z──RY(3.15)─╭●──RY(2.09)─╰Z──RY(4.01)─┤  <Z>
5: ──RY(0.00)─╰Z──RY(1.64)─╭●──RY(2.47)─╰Z──RY(3.55)─╭●──RY(3.51)─┤  <Z>
6: ──RY(0.00)─╭●──RY(3.11)─╰Z──RY(2.00)─╭●──RY(3.05)─╰Z──RY(2.38)─┤  <Z>
7: ──RY(0.00)─╰Z──RY(2.79)─╭●──RY(2.82)─╰Z──RY(2.06)─╭●──RY(2.78)─┤  <Z>
8: ──RY(0.00)──────────────╰Z──RY(2.96)──────────────╰Z──RY(3.60)─┤  <Z>


In [24]:
node(inputs,w,b)

tensor([0.80015126, 0.9828517 , 0.92534344, 0.55664437, 0.08115117,
        0.41864232, 0.41427645, 0.41553038, 0.87240161], requires_grad=True)

In [25]:
class QConv2D(tf.keras.layers.Layer):
    """
    2D Quantum convolution layer (e.g. spatial convolution over images).
    This layer creates a convolution kernel that is convolved 
    with the layer input to produce a tensor of outputs. Finally,
    `activation` is applied to the outputs as well.
    """

    def __init__(
            self,
            template = 'Basic',
            name='QConv2D',
    ):

        super(QConv2D, self).__init__(name=name)

        self.kernel_size = (3,3)
        self.strides = (2,2)
        
        node, shapes = get_node(template, args.n_layers, args.n_qubits, np.prod(self.kernel_size))
        self.conv_pqc = qml.qnn.KerasLayer(node, shapes, output_dim= args.n_qubits)
        
    def build(self, input_shape):
    
        self.iters, _ = convolution_iters(
            input_shape[1:3], self.kernel_size, self.strides, 'valid')
        

    def _convolution(self, input_tensor):

        s = self.strides
        k = self.kernel_size

        conv_out = []
        for i in range(self.iters[0]):
            for j in range(self.iters[1]):
                x = input_tensor[:, i * s[0]:i * s[0] + k[0], j *
                                 s[1]:j * s[1] + k[1]]
                x = tf.keras.layers.Flatten()(x)
                conv_out += [self.conv_pqc(x)]
              
        conv_out = tf.keras.layers.Concatenate(axis=1)(conv_out)
        conv_out = tf.keras.layers.Reshape((self.iters[0], self.iters[1], args.n_qubits))(conv_out)
        return conv_out

    def call(self, input_tensor):
  
        conv_out = [self._convolution(input_tensor[:, :, :, 0])]
        conv_out = tf.keras.layers.Concatenate(axis=-1)(conv_out)
        return conv_out

In [26]:
rlayer = QConv2D(template = args.template)
flayer = tf.keras.layers.Flatten()
clayer_2 = tf.keras.layers.Dense(2, activation="softmax")
model = tf.keras.models.Sequential([rlayer, flayer, clayer_2])

In [27]:
opt = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
model.compile(opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [28]:
dim = data.config()['input_dims']
in_shape = [args.batch_size] + list(dim)

x = tf.random.uniform((in_shape))
y = model(x)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 QConv2D (QConv2D)           multiple                  320       
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  164       
                                                                 
Total params: 484
Trainable params: 484
Non-trainable params: 0
_________________________________________________________________


In [29]:
fitting = model.fit(data.train_ds,
                    batch_size= args.batch_size,
                   epochs= args.epochs,
                   validation_data=data.val_ds,
                   shuffle=True,
                    callbacks = callbacks,
                   workers=4)

Epoch 1/8
8/8 [==============================] - 218s 27s/step - loss: 1.5626 - accuracy: 0.6146 - val_loss: 0.6502 - val_accuracy: 0.7833 - _timestamp: 1659854947.0000 - _runtime: 385.0000
Epoch 2/8
8/8 [==============================] - 228s 29s/step - loss: 0.6072 - accuracy: 0.8313 - val_loss: 0.7634 - val_accuracy: 0.8250 - _timestamp: 1659855175.0000 - _runtime: 613.0000
Epoch 3/8
8/8 [==============================] - 244s 31s/step - loss: 0.3982 - accuracy: 0.8917 - val_loss: 0.3680 - val_accuracy: 0.9000 - _timestamp: 1659855419.0000 - _runtime: 857.0000
Epoch 4/8
8/8 [==============================] - 469s 63s/step - loss: 0.2774 - accuracy: 0.9187 - val_loss: 0.3469 - val_accuracy: 0.9083 - _timestamp: 1659855887.0000 - _runtime: 1325.0000
Epoch 5/8
8/8 [==============================] - 226s 28s/step - loss: 0.1292 - accuracy: 0.9458 - val_loss: 0.2643 - val_accuracy: 0.9167 - _timestamp: 1659856114.0000 - _runtime: 1552.0000
Epoch 6/8
8/8 [==============================] -